In [4]:
from Tkinter import *
import math
import numpy as np

In [2]:
class globalCanvas(Frame):
    def get_canvas(self):
        return self.canvas
    
    def __init__(self, parent):
        Frame.__init__(self, parent)
        self.canvas = Canvas(self)
        self.parent = parent
        self.parent.title("Sudoku")
        self.pack(fill=BOTH, expand=1)

In [3]:
class SudokuUI():
    
    ''' getters and setters '''
    def get_gridCoord(self):
        return self.__gridCoord.copy()
    
    def get_initValue(self):
        return self.__initValue.copy()
        
    def get_solutionValue(self):
        return self.__solutionValue
    
    def get_x0(self):
        return self.__x0
    def set_x0(self, x0):
        if (x0 >= 0):
            self.__x0 = x0
    
    def get_y0(self):
        return self.__y0
    def set_y0(self, y0):
        if (y0 >= 0):
            self.__y0 = y0
    
    def get_length(self):
        return self.__length
    def set_length(self, value):
        self.__length = value
    
    ''' constructor '''
    def __init__(self, canvas):
        self.canvas = canvas
        self.__gridCoord = np.zeros((9,9), [('x', 'i4'), ('y', 'i4')])
        self.__initValue = np.zeros((9,9), int)
        self.__solutionValue = np.zeros((9,9), int)
        self.__x0 = 0
        self.__y0 = 0
        self.__length = 900
    
    ''' methods '''
    def import_problem(self, problemFile):
        importFile = open(problemFile, 'r')
        for j in xrange(9):
            line = importFile.readline()
            split = line.split()
            for i in xrange(9):
                self.__initValue[i,j] = int(split[i])
        #print self.__initValue
    
    def drawGrid(self):
        stride = self.__length/9
        
        for i in xrange(9):
            for j in xrange(9):
                self.__gridCoord[i,j]['x'] = self.__x0+(stride*i)
                self.__gridCoord[i,j]['y'] = self.__y0+(stride*j)
        #print self.__gridCoord

        for i in xrange(10):
            color = "blue" if i % 3 == 0 else "gray"
            self.canvas.create_line(self.__x0 + (i * stride), 
                                    self.__y0, 
                                    self.__x0 + (i * stride), 
                                    self.__y0 + self.__length, 
                                    fill=color)
            self.canvas.create_line(self.__x0, 
                                    self.__y0 + i * stride, 
                                    self.__x0 + self.__length, 
                                    self.__y0 + i * stride, 
                                    fill=color)
    
    def drawInitValues(self):
        stride = self.__length/9
        for i in xrange(9):
            for j in xrange(9):
                if self.__initValue[i,j] != 0: 
                    self.canvas.create_text(self.__gridCoord[i,j]['x']+stride/2,
                                            self.__gridCoord[i,j]['y']+stride/2,
                                            anchor=CENTER, font="Helvetica",
                                            text=self.__initValue[i,j],
                                            fill="black")
    def drawSolvedValues(self):
        stride = self.__length/9
        for i in xrange(9):
            for j in xrange(9):
                if self.__solutionValue[i,j] != 0: 
                    self.canvas.create_text(self.__gridCoord[i,j]['x']+stride/2,
                                            self.__gridCoord[i,j]['y']+stride/2,
                                            anchor=CENTER, font="Helvetica",
                                            text=self.__solutionValue[i,j],
                                            fill="green")
    
    def drawPossible(self, possibles):
        stride = self.__length/9
        for i in xrange(9):
            for j in xrange(9):
                possible_str = ""
                possible_num = 0
                for p in xrange(9):
                    if possibles[i][j][p] == True:
                        possible_num += 1
                        possible_str += str(p+1)
                        possible_str += ","
                if possible_num > 1:
                    self.canvas.create_text(self.__gridCoord[i,j]['x']+stride/2,
                                            self.__gridCoord[i,j]['y']+stride/2,
                                            anchor=CENTER, font="Helvetica",
                                            text=possible_str,
                                            fill="blue")
    
    # draws everything on the canvas all at once
    def draw(self):
        self.canvas.pack(fill=BOTH, expand=1)
    
    def clear(self):
        self.canvas.delete("all")

In [4]:
class SudokuAI():
    ''' private vars '''
    __workingValue = np.zeros((9,9), int)
    __possibleSpace = np.zeros((9,9,9), bool)
    for i in xrange(9):
        for j in xrange(9):
            for k in xrange(9):
                __possibleSpace[i,j,k] = True
    
    def get_workingValue(self):
        return self.__workingValue
    def get_possibleSpace(self):
        return self.__possibleSpace
    
    ''' constructor '''
    def __init__(self, problem_set):
        self.__workingValue = problem_set
    
    ''' methods '''
    def get_nonet(self, posx, posy):
        # nonet contains coordinates x,y for all the locations of a nonet containing posx, posy
        nonet = np.zeros(9, [('x', 'i4'), ('y', 'i4')])
        nonetPos = [int(math.floor(posx/3)),int(math.floor(posy/3))]
        for i in xrange(3):
            for j in xrange(3):
                nonet[i+3*j]['x'] = i+3*nonetPos[0]
                nonet[i+3*j]['y'] = j+3*nonetPos[1]
        return nonet
    
    def checkValidity(self, value, posx, posy):
        for i in xrange(9):
            if self.__workingValue[i,posy] == value:
                return False
        for j in xrange(9):
            if self.__workingValue[posx,j] == value:
                return False
        for k in xrange(9):
            if self.__workingValue[self.get_nonet(posx,posy)[k]['x'],self.get_nonet(posx,posy)[k]['y']] == value:
                return False
        return True
    
    def rule0(self):
        for i in xrange(9):
            for j in xrange(9):
                if self.__workingValue[i,j] != 0:
                    for k in xrange(9):
                        self.__possibleSpace[i,j,k] = False
    
    # checks all the possible values for each empty space
    # for the columns
    def rule1(self):
        #print type(self.__possibleSpace[:,0][0])
        # iterating through the cols
        for i in xrange(9):
            # iterating through the elements to find an existing value
            for j in xrange(9):
                if self.__workingValue[i,j] != 0:
                    # iterates through the elements again to find empty values
                    for jj in xrange(9):
                        if self.__workingValue[i,jj] == 0:
                            # remove the possibility of the existing element in the empty space
                            self.__possibleSpace[i,jj,self.__workingValue[i,j]-1] = False
        #print self.__possibleSpace
    
    # for the rows
    def rule2(self):
        #print type(self.__possibleSpace[:,0][0])
        # iterating through the rows
        for j in xrange(9):
            # iterating through the elements to find an existing value
            for i in xrange(9):
                if self.__workingValue[i,j] != 0:
                    # iterates through the elements again to find empty values
                    for ii in xrange(9):
                        if self.__workingValue[ii,j] == 0:
                            # remove the possibility of the existing element in the empty space
                            self.__possibleSpace[ii,j,self.__workingValue[i,j]-1] = False
        #print self.__possibleSpace
    
    def rule3(self):
        #print type(self.__possibleSpace[:,0][0])
        # iterating through the nonets
        for n in xrange(9):
            nonet = self.get_nonet(3*(n/3), 3*(n-3*(n/3)))
            for elem in xrange(9):
                if self.__workingValue[nonet[elem]['x'],nonet[elem]['y']] != 0:
                    for elem2 in xrange(9):
                        if self.__workingValue[nonet[elem2]['x'],nonet[elem2]['y']] == 0:
                            self.__possibleSpace[nonet[elem2]['x'],nonet[elem2]['y'],
                                                 self.__workingValue[nonet[elem]['x'],nonet[elem]['y']]-1] = False
        #print self.__possibleSpace
    
    def updateValues(self, displayValues):
        countUpdates = 0
        newValues = np.zeros((9,9), int)
        for i in xrange(9):
            for j in xrange(9):
                countPossible = 0
                ans = 0
                for k in xrange(9):
                    if self.__possibleSpace[i,j,k] == True:
                        countPossible += 1
                        ans = k
                if countPossible == 1:
                    countUpdates += 1
                    newValues[i,j] = ans+1
        self.__workingValue += newValues
        displayValues += newValues
        if countUpdates == 0:
            return True
        else:
            return False

In [5]:
def iterateCallBack (canvas, ai, env):
    # run ai rules
    env.clear()
    
    ai.rule0()
    ai.rule1()
    ai.rule2()
    ai.rule3()
    if ai.updateValues(env.get_solutionValue()) == False:
        env.drawGrid()
        env.drawInitValues()
        env.drawSolvedValues()
        env.drawPossible(ai.get_possibleSpace())
        env.draw()
    else:
        env.drawGrid()
        env.drawInitValues()
        env.drawSolvedValues()
        check = SudokuUI(canvas)
        check.set_x0(1000)
        check.set_y0(10)
        check.set_length(450)
        check.import_problem('easy1_s.txt')
        check.drawGrid()
        check.drawInitValues()
        check.draw()
        print "done"

def main():
  
    root = Tk()
    canvas = globalCanvas(root)
    environment = SudokuUI(canvas.get_canvas())
    environment.set_x0(10)
    environment.set_y0(10)
    environment.import_problem('easy1_p.txt')
    
    ai = SudokuAI(environment.get_initValue())
    b = Button(text="iterate", command=lambda: iterateCallBack(canvas.get_canvas(), ai, environment))
    b.pack()
    
    environment.drawGrid()
    environment.drawInitValues()
    #environment.drawPossible(ai.get_possibleSpace())
    environment.draw()
    w, h = root.winfo_screenwidth(), root.winfo_screenheight()
    root.geometry("%dx%d+0+0" % (w, h))
    
    while True:
        root.update_idletasks()
        root.update()

if __name__ == '__main__':
    main()

IOError: [Errno 2] No such file or directory: 'easy1_p.txt'

In [ ]:
grid = [[[c+10*x+100*y for c in xrange(1)] for x in xrange(9)] for y in xrange(5)]

r = grid[0]
def get_rows(row, data):
    rows = []
    for i in range (0, len(data)):
        col = data[i]
        rows.append(col[row])
    return rows

c = get_rows(0,grid)

print c
print r
print grid

In [35]:
npgrid = np.zeros((9,9), [('0','i4'), ('1', 'i4'),('2','i4'), ('3', 'i4'), ('4','i4'), ('5', 'i4'),('6','i4'), ('7', 'i4'), ('8', 'i4')])
for i in xrange(9):
    for j in xrange(9):
        for p in xrange(4):
            npgrid[i,j][p] = p+1
#print npgrid

In [33]:
a = possibleNums
npgrid = np.zeros((9,9), a)
'''for i in xrange(9):
    for j in xrange(9):
        for p in xrange(9):
            npgrid[i,j][p] = p+1+10*i'''
print npgrid[0,0].tolist()
print npgrid[0,0]

TypeError: data type not understood

In [7]:
def get_nonet(posx, posy):
    # nonet contains coordinates x,y for all the locations of a nonet containing posx, posy
    nonet = np.zeros(9, [('x', 'i4'), ('y', 'i4')])
    for i in xrange(3):
        for j in xrange(3):
            nonet[i+3*j]['x'] = i+3*posx
            nonet[i+3*j]['y'] = j+3*posy
    return nonet
for n in xrange(9):
    print str((n/3)) + str((n-3*(n/3)))
    print get_nonet((n/3), (n-3*(n/3)))

00
[(0, 0) (1, 0) (2, 0) (0, 1) (1, 1) (2, 1) (0, 2) (1, 2) (2, 2)]
01
[(0, 3) (1, 3) (2, 3) (0, 4) (1, 4) (2, 4) (0, 5) (1, 5) (2, 5)]
02
[(0, 6) (1, 6) (2, 6) (0, 7) (1, 7) (2, 7) (0, 8) (1, 8) (2, 8)]
10
[(3, 0) (4, 0) (5, 0) (3, 1) (4, 1) (5, 1) (3, 2) (4, 2) (5, 2)]
11
[(3, 3) (4, 3) (5, 3) (3, 4) (4, 4) (5, 4) (3, 5) (4, 5) (5, 5)]
12
[(3, 6) (4, 6) (5, 6) (3, 7) (4, 7) (5, 7) (3, 8) (4, 8) (5, 8)]
20
[(6, 0) (7, 0) (8, 0) (6, 1) (7, 1) (8, 1) (6, 2) (7, 2) (8, 2)]
21
[(6, 3) (7, 3) (8, 3) (6, 4) (7, 4) (8, 4) (6, 5) (7, 5) (8, 5)]
22
[(6, 6) (7, 6) (8, 6) (6, 7) (7, 7) (8, 7) (6, 8) (7, 8) (8, 8)]


clicked!
